# CS 584 :: Data Mining :: George Mason University :: Fall 2025


# Homework 3: Clustering

- **100 points [6% of your final grade]**
- **Due Sunday, November 16 by 11:59pm**

- *Goals of this homework:* (1) implement your K-means model; (2) implement K-means++; (3) Apply PCA to K-Means.

- *Submission instructions:* for this homework, you should submit your notebook file to **Canvas** (look for the homework 3 assignment there). Please name your submission **FirstName_Lastname_hw3.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw3.ipynb**. Your notebook should be fully executed so that we can see all outputs.

## Part 1: K-Means (60 points)

In this part, you will implement your own K-means algorithm to conduct clustering on handwritten digit images. In this homework, we will still use the handwritten digit image dataset we have already used in previous homework. However, since clustering is unsupervised learning, which means we do not use the label information for model training anymore. So, here, we will only use the testing data stored in the "test.txt" file.

First, let's load the data by executing the following code:

In [27]:
import numpy as np

test = np.loadtxt("test.txt", delimiter=',')
test_features = test[:, 1:]
test_labels = test[:, 0]
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing label matrix: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing label matrix: shape (10000,)


Now, it's time for you to implement your own K-means algorithm. First, please write your code to build your K-means model using the image data with **K = 10**, and **Euclidean distance**.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn**

**Note: you need to decide when to stop the iteration.**

In [2]:
# Write your code here
from tqdm import tqdm

def euc_dist(data,cluster):
    return np.sqrt(np.sum((data-cluster)**2))

def cent_comp(data,cluster,centroid,k):
    d=data.shape[1]
    new_centroids=np.empty((k,d), dtype=float)
    for i in range(k):
        points=(data[cluster==i])
        if(points.size==0):
            new_centroids[i]=centroid[i]
        else:
            new_centroids[i]=points.mean(axis=0)
    return new_centroids

centroid_idx=np.random.permutation(test_features.shape[0])[:10]
cent=np.array([test_features[idx] for idx in centroid_idx],dtype=float)
clust=np.empty(len(test_features), int)
def k_means(k, iter,tol,centroid,cluster):
    
    #print(euc_dist(centroid[1],test_features[0])) # now centroid is initialized as list of 10 random features from test features
    
    for j in tqdm(range(iter)):
        
        for i,data in enumerate(test_features):
            k_dist=euc_dist(data,centroid[0])
            k_cluster=0
            for idx in range(1, len(centroid)):
                i_dist=euc_dist(data,centroid[idx])
                if(i_dist<k_dist):
                    k_dist=i_dist
                    k_cluster=idx
            cluster[i]=k_cluster
        
        x=test_features
        new_centroid=cent_comp(x,cluster,centroid,k)
        diff=np.sum(abs(new_centroid-centroid))
        #diff=euc_dist(new_centroid,centroid)
        if(diff<tol):
            centroid=new_centroid
            break
        #print(f'distance between old/new: {euc_dist(new_centroid,centroid)}')
        centroid=new_centroid
        #print(f'test {j}')
    return centroid,cluster
cent,clust=k_means(10,100,1e-10,cent,clust)

print('done')


 65%|████████████████████████████████████████████████████▋                            | 65/100 [00:56<00:30,  1.15it/s]

done


Next, you need to calculate and print the **square root** of **S**um of **S**quared **E**rror (SSE) of each cluster generated by your K-means algorithm. Then, print out the averaged square root of SSE over all clusters.

**Note: The value of "square root of SSE" can diverge significantly. The expected value range is 10000~100000.**

In [3]:
# Write your code here
k=cent.shape[0]
total=0
for i in range(k):
    i_pts=test_features[clust==i]
    sse=np.sqrt(np.sum((i_pts-cent[i])**2))
    total+=sse
    print(f'For cluster {i}: {sse}')
print(f'\nAverage SSE for all clusters is: {total/k}')
    

For cluster 0: 52623.02270738569
For cluster 1: 61252.46806774508
For cluster 2: 52415.45838948367
For cluster 3: 49370.00291654311
For cluster 4: 58800.54975634594
For cluster 5: 50226.538306916096
For cluster 6: 50908.591569024975
For cluster 7: 34516.27744916376
For cluster 8: 49669.45108077166
For cluster 9: 37193.69839221044

Average SSE for all clusters is: 49697.60586355904


Then, please have a look on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure, and use this function to print out the homogeneity, completeness, and v-measure of your K-means model.

**Note: The values of homogeneity, completeness, and v-measure are expected to be >0.48**

In [4]:
# Write your code here
from sklearn.metrics import homogeneity_completeness_v_measure

hcv=[]
hcv=homogeneity_completeness_v_measure(test_labels, clust)
print(f'Homogeneity: {hcv[0]}\nCompleteness: {hcv[1]}\nV-Measure: {hcv[2]}')

Homogeneity: 0.49912264364737735
Completeness: 0.5025963389299022
V-Measure: 0.500853468362487


## Part 2: K-Means++ (30 points)

Ok, now you already have a good model. But can you further improve it? In the next cell, please implement the K-means++ model introduced in the lecture.

**Note: Everything else is the same as Part1, i.e., K=10, and you need to use Euclidean distance.**

In [14]:
# reset centroid/cluster arrays
cent=np.zeros((10,test_features.shape[1]))
clust=np.empty(len(test_features), int)

#initialize first cluster w/ random point in data, and min_dist array with inf, to capture min distances between point and centroid
min_dist=np.full(test_features.shape[0],np.inf)
rand_idx=np.random.choice(test_features.shape[0])
cent[0]=test_features[rand_idx]

k=10
#loop for k++ initialization
for j in range(1,k):
    for i in range(len(test_features)):
        d=euc_dist(test_features[i],cent[j-1])
        if d<min_dist[i]:
            min_dist[i]=d
    probs=min_dist/np.sum(min_dist)
    idx=np.random.choice(len(test_features),p=probs)
    cent[j]=test_features[idx]
print(len(cent))

#run k-means with empty cluster and newly k++ initialized centroids
cent,clust=k_means(10,100,1e-10,cent,clust)

print('done')


10


 51%|█████████████████████████████████████████▎                                       | 51/100 [00:44<00:42,  1.14it/s]

done


In the next cell, use sklearn.metrics.homogeneity_completeness_v_measure() to print out the homogeneity, completeness, and v-measure of your K-means++ model.

In [15]:
# Write your code here
hcv_2=homogeneity_completeness_v_measure(test_labels, clust)
print(hcv_2[0],hcv_2[1],hcv_2[2])

0.5128245265444459 0.5185967310678528 0.5156944771393042


## Part 2: Dimension Reduction by PCA (10 points)

Last, in this part, please use PCA to reduce the feature dimension here. And then, apply your K-Means code to the reduced data and report homogeneity, completeness, and v-measure.

**Note: You need to consider the reduced dimension m of PCA as a hyper-parameter to tune. That is, you need to try different m and measure the corresponding clustering performance. At the end, you need to report the best m and clustering performance.**

**Note: Everything else is the same as Part1, i.e., K=10, and you need to use Euclidean distance.**

**Note: You can reuse your own code of PCA from your HW1, or you can directly use the PCA model from sklearn -- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html**

In [28]:
from sklearn.decomposition import PCA

#150 features provided good metrics 
pca=PCA(200)
pca.fit(test_features)
reduced_features=pca.transform(test_features)
print(reduced_features.shape[0])
test_features=reduced_features

#reset centroid and cluster to empty and a random permutation of features beased on reduced feature set
centroid_idx=np.random.permutation(test_features.shape[0])[:k]
cent=test_features[centroid_idx].astype(float)
clust=np.empty(len(test_features), int)
                                
cent,clust=k_means(10,100,1e-10,cent,clust)
print('done')

10000


 81%|█████████████████████████████████████████████████████████████████▌               | 81/100 [00:56<00:13,  1.42it/s]

done


In the next cell, use sklearn.metrics.homogeneity_completeness_v_measure() to print out the homogeneity, completeness, and v-measure of your K-means model.

**Note: The values of homogeneity, completeness, and v-measure are expected to be >0.48**

In [29]:
# Write your code here
hcv_3=homogeneity_completeness_v_measure(test_labels, clust)
print(hcv_3[0],hcv_3[1],hcv_3[2])

0.5213249904417651 0.5359245861183178 0.5285239851212393
